In [ ]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from shapely.geometry import Polygon, box
from datetime import datetime
import logging
from ultralytics import YOLO
from tqdm.auto import tqdm

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class TramTracker:
    def __init__(self, area_coordinates_file='area_coordinates.txt'):
        # Load Area A coordinates
        self.area_a = self.load_area_a(area_coordinates_file)
        self.results = []
        self.arrivals = []  # New list to store each arrival
        self.current_arrival = None  # Track current arrival
        
    def load_area_a(self, coordinates_file):
        """Load and create Area A polygon"""
        try:
            points = np.loadtxt(coordinates_file, dtype=np.int32)
            return Polygon(points)
        except Exception as e:
            logger.error(f"Error loading area coordinates: {e}")
            raise
    
    def is_in_area_a(self, bbox):
        """
        Check if tram is in Area A
        bbox: [x1, y1, x2, y2] from YOLO detection
        """
        try:
            tram_box = box(bbox[0], bbox[1], bbox[2], bbox[3])
            intersection_area = self.area_a.intersection(tram_box).area
            tram_area = tram_box.area
            
            # Consider tram in Area A if more than 50% of its area intersects
            return intersection_area / tram_area > 0.5
        except Exception as e:
            logger.error(f"Error checking area intersection: {e}")
            return False
    
    def extract_timestamp(self, filename):
        """Extract timestamp from filename"""
        try:
            # Assuming filename format: frame_YYYYMMDD_HHMMSS.png
            parts = filename.stem.split('_')
            date_str = parts[-2]
            time_str = parts[-1]
            return datetime.strptime(f"{date_str}_{time_str}", "%Y%m%d_%H%M%S")
        except Exception as e:
            logger.error(f"Error extracting timestamp from {filename}: {e}")
            raise

    def process_frame(self, image_path, yolo_model, save_debug_frames=False):
        """Process a single frame"""
        try:
            # Get timestamp
            timestamp = self.extract_timestamp(Path(image_path))

            # Run YOLO detection
            results = yolo_model.predict(
                source=str(image_path),
                conf=0.5,
                verbose=False
            )
            
            # Track if tram is in Area A
            tram_detected = False
            tram_in_area_a = False
            bbox = None  # Initialize bbox as None
            
            # Process detections
            if len(results) > 0:  # If there are any detections
                result = results[0]  # Get the first result
                
                for box in result.boxes:
                    # Get class id and confidence
                    class_id = int(box.cls[0].item())
                    confidence = box.conf[0].item()
                    
                    # Assuming class_id 0 is tram (adjust if needed)
                    if class_id == 0 and confidence > 0.5:
                        tram_detected = True
                        # Get bounding box coordinates
                        bbox = box.xyxy[0].cpu().numpy()
                        
                        # Check if tram is in Area A
                        tram_in_area_a = self.is_in_area_a(bbox)
                        break  # Assuming we only care about the first tram detected
            
            # Always save debug frame, whether tram is detected or not
            if save_debug_frames:
                self.save_debug_frame(str(image_path), bbox, tram_in_area_a, timestamp, image_path)
            
            # Store result
            self.results.append({
                'timestamp': timestamp,
                'tram_detected': tram_detected,
                'in_area_a': tram_in_area_a
            })
            
            logger.debug(f"Processed {image_path}: Tram detected: {tram_detected}, In Area A: {tram_in_area_a}")
            
        except Exception as e:
            logger.error(f"Error processing frame {image_path}: {e}")
            
    def save_debug_frame(self, image_path, bbox, in_area_a, timestamp, original_path):
        """Save debug frame with visualization"""
        # Read the image
        debug_image = cv2.imread(image_path)
        
        # Draw Area A
        area_points = np.array(self.area_a.exterior.coords[:-1], dtype=np.int32)
        cv2.polylines(debug_image, [area_points], True, (0, 255, 0), 2)
        
        # Add timestamp
        height, _ = debug_image.shape[:2]
        cv2.putText(debug_image, 
                   timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                   (10, height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 
                   .5, 
                   (255, 255, 255), 
                   2)
        
        if bbox is not None:  # If tram is detected
            # Draw tram bounding box
            x1, y1, x2, y2 = map(int, bbox)
            color = (0, 255, 0) if in_area_a else (0, 0, 255)
            cv2.rectangle(debug_image, (x1, y1), (x2, y2), color, 2)
            
            # Add text
            status = "In Area A" if in_area_a else "Outside Area A"
            cv2.putText(debug_image, status, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        else:
            # Add "No Tram Detected" text
            cv2.putText(debug_image, 
                       "No Tram Detected", 
                       (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       1, 
                       (0, 0, 255), 
                       2)
        
        # Save debug frame
        debug_path = Path('debug_frames') / f"debug_{Path(original_path).stem}.jpg"
        debug_path.parent.mkdir(exist_ok=True)
        cv2.imwrite(str(debug_path), debug_image)

    def calculate_duration(self):
        """Calculate durations for all tram arrivals"""
        if not self.results:
            return pd.DataFrame()
            
        arrivals_data = []
        entry_time = None
        arrival_count = 0
        
        for result in self.results:
            if result['in_area_a'] and entry_time is None:
                # Tram just entered Area A
                entry_time = result['timestamp']
                arrival_count += 1
            elif not result['in_area_a'] and entry_time is not None:
                # Tram just left Area A
                exit_time = result['timestamp']
                duration = (exit_time - entry_time).total_seconds()
                
                arrivals_data.append({
                    'Arrival No.': arrival_count,
                    'Enter Time': entry_time.strftime('%H:%M:%S'),
                    'Exit Time': exit_time.strftime('%H:%M:%S'),
                    'Duration (Seconds)': duration
                })
                entry_time = None
        
        # Handle case where tram is still in Area A at the end
        if entry_time is not None:
            exit_time = self.results[-1]['timestamp']
            duration = (exit_time - entry_time).total_seconds()
            arrivals_data.append({
                'Arrival No.': arrival_count,
                'Enter Time': entry_time.strftime('%H:%M:%S'),
                'Exit Time': exit_time.strftime('%H:%M:%S'),
                'Duration (Seconds)': duration
            })
        
        # Create DataFrame
        return pd.DataFrame(arrivals_data)

def generate_report(results, arrivals_df):
    """Generate a detailed report of the analysis"""
    report_path = Path('analysis_report.txt')
    
    with report_path.open('w') as f:
        f.write("Tram Analysis Report\n")
        f.write("===================\n\n")
        
        f.write(f"Total Frames Analyzed: {len(results)}\n\n")
        
        f.write("Tram Arrivals Summary:\n")
        f.write("---------------------\n")
        f.write(arrivals_df.to_string())
        f.write("\n\n")
        
        f.write("Detailed Timeline:\n")
        f.write("----------------\n")
        
        in_area = False
        for result in results:
            if result['in_area_a'] != in_area:
                timestamp = result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
                status = "entered" if result['in_area_a'] else "left"
                f.write(f"{timestamp}: Tram {status} Area A\n")
                in_area = result['in_area_a']

# Initialize your model
model_path = "build_custom_model/runs/train/ayana_tram_stop_detection/weights/best.pt"
yolo_model = YOLO(model_path)

# Initialize TramTracker
tracker = TramTracker('area_coordinates.txt')

image_folder = Path('dataset/captured_frames/2024-12-02')
image_subfolders = [f for f in image_folder.glob('*') if f.is_dir()]
image_files = []
for image_subfolder in image_subfolders:
    image_files.extend(sorted(image_subfolder.glob('ayana_tram_stop_*.jpg')))
image_files = sorted(image_files)

# Process each frame
for image_file in tqdm(image_files, desc="Processing frames"):
    tracker.process_frame(image_file, yolo_model, save_debug_frames=True)

# Get arrivals DataFrame
arrivals_df = tracker.calculate_duration()

# Print results
print("\nTram Arrivals Summary:")
print(arrivals_df)

# Save to CSV
arrivals_df.to_csv('tram_arrivals.csv', index=False)

# Generate detailed report with arrivals information
generate_report(tracker.results, arrivals_df)

Processing frames:   0%|          | 0/56806 [00:00<?, ?it/s]

In [ ]:
import os
target_date = '2024-12-02'
path = '/Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/dataset/captured_frames/' + target_date

incorrect = []
for subpath in os.listdir(path):
    subpath_hour = subpath.split('_')[-1][:2]
    subpath = os.path.join(path, subpath)
    if os.path.isdir(subpath):
        image_folder = Path(subpath)
        image_files = sorted(image_folder.glob('ayana_tram_stop_*.jpg'))[:20]
        for img in image_files:
            # img_hour = img.as_posix().split('/')[-1].split('_')[-1].split('.')[0][:2]
            # if img_hour != subpath_hour:
            #     incorrect.append(subpath)
            img_year = img.as_posix().split('/')[-1].split('_')[-2][:4]
            if img_year != '2024':
                incorrect.append(subpath)
            
incorrect = sorted(list(set(incorrect)))
incorrect

In [11]:
import os
from pathlib import Path

# Your existing code
subpath = '/Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/dataset/captured_frames/2024-12-02/ayana_tram_stop_20241202_080033'
image_folder = Path(subpath)
image_files = sorted(image_folder.glob('ayana_tram_stop_*.jpg'))

# Renaming logic
for file_path in image_files:
    # Get the old filename
    old_name = file_path.name
    
    # Create new filename by replacing '00' with '08' in the hour position
    new_name = old_name.replace('_20141202_', '_20241202_')
    
    # Create full paths
    old_path = file_path
    new_path = file_path.parent / new_name
    
    # # Rename the file
    os.rename(old_path, new_path)
    # print(f'Renamed: {old_name} → {new_name}')